In [ ]:
# Title

In [1]:
import numpy as np
from numpy import pi, r_
import matplotlib.pyplot as plt
from scipy import optimize
import pandas as pd
from matplotlib import rc
import matplotlib.ticker as mtick
import re
import sys
import os
import math
#import decimal
from scipy.optimize import minimize
import collections


In [2]:
# 1. read in atom names, atom nambers, residue numbers from topol.top
# and marking backbone(bb) atoms

# Using readline() 
file1 = open('/Users/psz898/OneDrive - Københavns Universitet/Documents/SCIENCE/UCPH/info_comp/parent_bb_test/input/topol_no4957.top', 'r') 
count = 0

regex = re.compile("\s+(\d+)\s+\w+\s+(\d+)\s+\w{3}\s+([N|C|O]A?)\s+")
regexH = re.compile("\s+(\d+)\s+\w+\s+(\d+)\s+\w{3}\s+(H)\d*\s+")
regexOC = re.compile("\s+(\d+)\s+\w+\s+(\d+)\s+\w{3}\s+(OC)\d+\s+")
regexALL = re.compile("^\s+(\d+)\s+\w+\*?\d*\s+(\d+)\s+\w{3}\s+\w+\d*\s+\d+\s+-?0.\d+\s+\d+.?\d*")
#    894          N     55    ASP      N    894    -0.5163      14.01

bbatoms = []
bbatomNames = []
bbatomsRes = []

Allatoms = []
AllatomsRes = []

while True: 
    count += 1
  
    # Get next line from file 
    line = file1.readline() 
  
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
    resultALL = regexALL.search(line)
    result = regex.search(line)
    resultH = regexH.search(line)
    resultOC = regexOC.search(line)
    
    
    #print("Line{}: {}".format(count, line.strip())) 
    #if resultALL or result or resultH or resultOC:
    if resultALL:
        Allatoms.append(int(resultALL.group(1)))        
        AllatomsRes.append(int(resultALL.group(2)))
        #print (line, resultALL.group(1))
    if result:
        #print (result.group(1), result.group(2))
        bbatoms.append(result.group(1))
        bbatomNames.append(result.group(3))
        bbatomsRes.append(int(result.group(2)))        
    if resultH:
        bbatoms.append(resultH.group(1))
        bbatomNames.append(resultH.group(3))
        bbatomsRes.append(int(resultH.group(2)))
    if resultOC:
        bbatoms.append(resultOC.group(1))
        bbatomNames.append(resultOC.group(3))
        bbatomsRes.append(int(resultOC.group(2)))

        
file1.close() 

bbatoms_arr = np.array(bbatoms) 
bbatomsInt = bbatoms_arr.astype(np.int) 

# dictionary: Allatoms - keys;  AllatomsRes - values
# new_dict = dict(zip(keys, values))
atoms_res_dict = dict(zip(Allatoms, AllatomsRes))
# print (atoms_res_dict[78])

# print (bbatoms)
# print (bbatomNames)
# print (bbatomsRes)

# print (len(AllatomsRes))
# print ("")
# print (AllatomsRes)

# print (len(Allatoms))
# print ("")
# print (Allatoms)


In [3]:
# 2. reading in info about bonds (b), angles (a), dihedral (d) for which entropy was calculated
# marking backbone b, a, d
# marking/sorting b,a,d per residue: creating dictionaries bonds_res_dict, etc.
file2 = open('/Users/psz898/OneDrive - Københavns Universitet/Documents/SCIENCE/UCPH/info_comp/parent_bb_test/input/PARENT-master_shuf_MIE_topology.txt', 'r') 
count = 0


# 1004 1015 1017 bonds
# 994 1001 1003 1004 angles 
# 1011 1003 1015 1017 1018 0 -1 dihedrals
# 1012 1003 1015 1017 1019 0 1011 dihedrals

re_bonds = re.compile("^(\d+)\s+(\d+)\s+(\d+)\n")
re_angles = re.compile("^(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\n")
re_dihedrals = re.compile("^(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+([-1|0|1])\s+(-?\d+)\n")

np.bbB = []
np.bbA = []
np.bbD = []

np.ResinBonds = []
np.keys_res_bonds = []
np.ResinBonds1 = []
np.ResinBonds2 = []

np.ResinAngles = []
np.keys_res_angles = []
np.ResinAngles1 = []
np.ResinAngles2 = []
np.ResinAngles3 = []

np.ResinDihedrals = []
np.keys_res_dihedrals = []
np.ResinDihedrals1 = []
np.ResinDihedrals2 = []
np.ResinDihedrals3 = []
np.ResinDihedrals4 = []


while True: 
    count += 1
  
    # Get next line from file 
    line = file2.readline() 
  
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
    result_b = re_bonds.search(line)
    result_a = re_angles.search(line)
    result_d = re_dihedrals.search(line)
    
# define participating residues for each b/a/d
# create dictionaries: key - bonds/ang/dih, values - <bonds'/ang/dih ref residue>    
# create rules to define a ref residue if b/a/d is shared between dif residues

    if result_b:
        np.keys_res_bonds.append(int(result_b.group(1))) 
        np.ResinBonds.append(atoms_res_dict[int(result_b.group(2))])  
        #np.ResinBonds1.append(atoms_res_dict[int(result_b.group(2))])
        #np.ResinBonds2.append(atoms_res_dict[int(result_b.group(3))])
        
    if result_a:
        np.keys_res_angles.append(int(result_a.group(1))) 
        if atoms_res_dict[int(result_a.group(2))]==atoms_res_dict[int(result_a.group(3))] or atoms_res_dict[int(result_a.group(2))]==atoms_res_dict[int(result_a.group(4))]:
            np.ResinAngles.append(atoms_res_dict[int(result_a.group(2))])   
        if atoms_res_dict[int(result_a.group(3))]==atoms_res_dict[int(result_a.group(4))] and atoms_res_dict[int(result_a.group(2))]!=atoms_res_dict[int(result_a.group(4))]:
            np.ResinAngles.append(atoms_res_dict[int(result_a.group(3))]) 
        
        #np.ResinAngles1.append(atoms_res_dict[int(result_a.group(2))])
        #np.ResinAngles2.append(atoms_res_dict[int(result_a.group(3))])
        #np.ResinAngles3.append(atoms_res_dict[int(result_a.group(4))])

    if result_d:
        np.keys_res_dihedrals.append(int(result_d.group(1))) 
        temp1 = atoms_res_dict[int(result_d.group(2))]
        temp2 = atoms_res_dict[int(result_d.group(3))]
        temp3 = atoms_res_dict[int(result_d.group(4))]
        temp4 = atoms_res_dict[int(result_d.group(5))]
        #print (temp1, temp2, temp3, temp4)
        
        if temp1==temp2==temp3!=temp4 or temp1==temp2==temp4!=temp3 or temp1==temp3==temp4!=temp2:
            np.ResinDihedrals.append(atoms_res_dict[int(result_d.group(2))])
        if temp2==temp3==temp4!=temp1:
            np.ResinDihedrals.append(atoms_res_dict[int(result_d.group(3))])
        if temp1==temp2 and temp3==temp4 and temp2!=temp3:
            np.ResinDihedrals.append(atoms_res_dict[int(result_d.group(2))])
        if temp1==temp2==temp3==temp4:
            np.ResinDihedrals.append(atoms_res_dict[int(result_d.group(2))])
            
        #np.ResinDihedrals1.append(atoms_res_dict[int(result_d.group(2))])
        #np.ResinDihedrals2.append(atoms_res_dict[int(result_d.group(3))])
        #np.ResinDihedrals3.append(atoms_res_dict[int(result_d.group(4))])
        #np.ResinDihedrals4.append(atoms_res_dict[int(result_d.group(5))])


    if result_b and result_b.group(2) in bbatoms and result_b.group(3) in bbatoms: 
        #print (result_b.group(1),result_b.group(2),result_b.group(3))
        
        ##### to check that indeed atom numbers in the bond list correspond to bb atoms in topol.top file 
        
        #atom1address = np.where(bbatomsInt == int(result_b.group(2)))
        #atom2address = np.where(bbatomsInt == int(result_b.group(3)))
        #a = np.asscalar(atom1address[0])
        #b = np.asscalar(atom2address[0])
        #print(a, b)
        #print(bbatomNames[a],bbatomNames[b])
        
        ##### to check... end
        
        np.bbB.append(int(result_b.group(1)))       

        
        
    if result_a and result_a.group(2) in bbatoms and result_a.group(3) in bbatoms and result_a.group(4) in bbatoms: 
        
        #print (result_a.group(1),result_a.group(2),result_a.group(3),result_a.group(4))   
        
        np.bbA.append(int(result_a.group(1)))  
      
    if result_d and result_d.group(2) in bbatoms and result_d.group(3) in bbatoms and result_d.group(4) in bbatoms and result_d.group(5) in bbatoms: 
        
        #print (result_a.group(1),result_a.group(2),result_a.group(3),result_a.group(4))   
        
        np.bbD.append(int(result_d.group(1)))  
    
    
    
file2.close() 

#print (len(np.keys_res_dihedrals))
#print(len(np.ResinDihedrals))

# new_dict = dict(zip(keys, values))
bonds_res_dict = dict(zip(np.keys_res_bonds, np.ResinBonds))  
angles_res_dict = dict(zip(np.keys_res_angles, np.ResinAngles))  
dihedrals_res_dict = dict(zip(np.keys_res_dihedrals, np.ResinDihedrals))  

#print (bonds_res_dict)

# print (len(np.bbB))
# print (len(np.bbA))
# print (len(np.bbD))

# print (np.bbB)
# print (np.bbA)
# print (np.bbD)

# print (np.ResinBonds1)
# print (np.ResinBonds2)
# print ("")
# print (np.ResinAngles1)
# print (np.ResinAngles2)
# print (np.ResinAngles3)

# print (np.ResinDihedrals4)


In [4]:
# 3. summing up b.a.d entropy for each residue using dictionaries

file3 = open('/Users/psz898/OneDrive - Københavns Universitet/Documents/SCIENCE/UCPH/info_comp/parent_bb_test/input/PARENT-master_shuf_MIE.txt', 'r') 
count = 0

# bond 1   -4.11054278058
# angle 397   -1.31856147262
# dihedral 255   -1.12024112645
# bond-bond 1  8   -5.3416134817   0.00243253522582
# bond-angle 500  876   -5.20099574171   0.00283904511501
# bond-diheral 500  876   -5.20099574171   0.00283904511501
# angle-angle 500  876   -5.20099574171   0.00283904511501
# angle-diheral 500  876   -5.20099574171   0.00283904511501
# dihedral-dihedral 1017  1021   -1.3292625302   0.00239929713761

e_bonds = re.compile("^bond\s+(\d+)\s+(-?\d+.\d+)\n")
e_angles = re.compile("^angle\s+(\d+)\s+(-?\d+.\d+)\n")
e_dihedrals = re.compile("^dihedral\s+(\d+)\s+(-?\d+.\d+)\n")

# e_bond_bond = re.compile("^bond-bond\s+(\d+)\s+(\d+)\s+(-?\d+.\d+)\s+-?\d+.\d+\n")
# e_bond_angle = re.compile("^bond-angle\s+(\d+)\s+(\d+)\s+(-?\d+.\d+)\s+-?\d+.\d+\n")
# e_bond_dihedral = re.compile("^bond-dihedral\s+(\d+)\s+(\d+)\s+(-?\d+.\d+)\s+-?\d+.\d+\n")

# e_angle_angle = re.compile("^angle-angle\s+(\d+)\s+(\d+)\s+(-?\d+.\d+)\s+-?\d+.\d+\n")
# e_angle_dihedral = re.compile("^angle-dihedral\s+(\d+)\s+(\d+)\s+(-?\d+.\d+)\s+-?\d+.\d+\n")

# e_dihedral_dihedral = re.compile("^dihedral-dihedral\s+(\d+)\s+(\d+)\s+(-?\d+.\d+)\s+-?\d+.\d+\n")


bonds_entropy_res_bb = {}
bonds_entropy_res_rest = {}
bonds_entropy_res_all = {}

angles_entropy_res_bb = {}
angles_entropy_res_rest = {}
angles_entropy_res_all = {}

dihedrals_entropy_res_bb = {}
dihedrals_entropy_res_rest = {}
dihedrals_entropy_res_all = {}


while True: 
    count += 1
  
    # Get next line from file 
    line = file3.readline() 
  
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
        
    result_bonds = e_bonds.search(line)
    result_angles = e_angles.search(line)
    result_dihedrals = e_dihedrals.search(line)
    
    if result_bonds:
        some_res = bonds_res_dict[int(result_bonds.group(1))]      
        bonds_entropy_res_all.setdefault(some_res, []).append(float(result_bonds.group(2)))        
    if result_bonds and int(result_bonds.group(1)) in np.bbB:
        some_res = bonds_res_dict[int(result_bonds.group(1))]      
        bonds_entropy_res_bb.setdefault(some_res, []).append(float(result_bonds.group(2)))       
    if result_bonds and int(result_bonds.group(1)) not in np.bbB:
        some_res = bonds_res_dict[int(result_bonds.group(1))]      
        bonds_entropy_res_rest.setdefault(some_res, []).append(float(result_bonds.group(2)))

    if result_angles:
        some_res = angles_res_dict[int(result_angles.group(1))]      
        angles_entropy_res_all.setdefault(some_res, []).append(float(result_angles.group(2)))        
    if result_angles and int(result_angles.group(1)) in np.bbA:
        some_res = angles_res_dict[int(result_angles.group(1))]      
        angles_entropy_res_bb.setdefault(some_res, []).append(float(result_angles.group(2)))
    if result_angles and int(result_angles.group(1)) not in np.bbA:
        some_res = angles_res_dict[int(result_angles.group(1))]      
        angles_entropy_res_rest.setdefault(some_res, []).append(float(result_angles.group(2)))

    if result_dihedrals:
        some_res = dihedrals_res_dict[int(result_dihedrals.group(1))]      
        dihedrals_entropy_res_all.setdefault(some_res, []).append(float(result_dihedrals.group(2))) 
    if result_dihedrals and int(result_dihedrals.group(1)) in np.bbD:
        some_res = dihedrals_res_dict[int(result_dihedrals.group(1))]      
        dihedrals_entropy_res_bb.setdefault(some_res, []).append(float(result_dihedrals.group(2)))
    if result_dihedrals and int(result_dihedrals.group(1)) not in np.bbD:
        some_res = dihedrals_res_dict[int(result_dihedrals.group(1))]      
        dihedrals_entropy_res_rest.setdefault(some_res, []).append(float(result_dihedrals.group(2)))
        
        
file3.close 

#bonds_entropy_res_sum = {k: [sum(bonds_entropy_res[k])] for k in bonds_entropy_res.keys()}
bonds_entropy_res_all_sum = {k: sum(bonds_entropy_res_all[k]) for k in bonds_entropy_res_all.keys()}
bonds_entropy_res_all_total_sum = sum(bonds_entropy_res_all_sum.values())

bonds_entropy_res_bb_sum = {k: sum(bonds_entropy_res_bb[k]) for k in bonds_entropy_res_bb.keys()}
bonds_entropy_res_bb_total_sum = sum(bonds_entropy_res_bb_sum.values())

bonds_entropy_res_rest_sum = {k: sum(bonds_entropy_res_rest[k]) for k in bonds_entropy_res_rest.keys()}
bonds_entropy_res_rest_total_sum = sum(bonds_entropy_res_rest_sum.values())


angles_entropy_res_all_sum = {k: sum(angles_entropy_res_all[k]) for k in angles_entropy_res_all.keys()}
angles_entropy_res_all_total_sum = sum(angles_entropy_res_all_sum.values())

angles_entropy_res_bb_sum = {k: sum(angles_entropy_res_bb[k]) for k in angles_entropy_res_bb.keys()}
angles_entropy_res_bb_total_sum = sum(angles_entropy_res_bb_sum.values())

angles_entropy_res_rest_sum = {k: sum(angles_entropy_res_rest[k]) for k in angles_entropy_res_rest.keys()}
angles_entropy_res_rest_total_sum = sum(angles_entropy_res_rest_sum.values())


dihedrals_entropy_res_all_sum = {k: sum(dihedrals_entropy_res_all[k]) for k in dihedrals_entropy_res_all.keys()}
dihedrals_entropy_res_all_total_sum = sum(dihedrals_entropy_res_all_sum.values())

dihedrals_entropy_res_bb_sum = {k: sum(dihedrals_entropy_res_bb[k]) for k in dihedrals_entropy_res_bb.keys()}
dihedrals_entropy_res_bb_total_sum = sum(dihedrals_entropy_res_bb_sum.values())

dihedrals_entropy_res_rest_sum = {k: sum(dihedrals_entropy_res_rest[k]) for k in dihedrals_entropy_res_rest.keys()}
dihedrals_entropy_res_rest_total_sum = sum(dihedrals_entropy_res_rest_sum.values())


print ("bonds bb:", bonds_entropy_res_bb_total_sum)
print ("bonds rest:", bonds_entropy_res_rest_total_sum)
print ("bonds all:", bonds_entropy_res_all_total_sum)
print("")
print ("angles bb:", angles_entropy_res_bb_total_sum)
print ("angles rest:", angles_entropy_res_rest_total_sum)
print ("angles all:", angles_entropy_res_all_total_sum)
print("")
print ("dihedrals bb:", dihedrals_entropy_res_bb_total_sum)
print ("dihedrals rest:", dihedrals_entropy_res_rest_total_sum)
print ("dihedrals all:", dihedrals_entropy_res_all_total_sum)


bonds bb: -602.4777793962099
bonds rest: -2129.835773291401
bonds all: -2732.3135526876113

angles bb: -503.99462658959783
angles rest: -1006.3217164066428
angles all: -1510.316342996241

dihedrals bb: -89.98577468117648
dihedrals rest: -341.48755185666886
dihedrals all: -431.4733265378453


In [5]:
# function to get MI negative. Is used to sum entropy with negative of MI (istead of subtraction of MI)
def negativedict(my_data):
    my_data.update((x , y*-1)for x, y in my_data.items())
    return my_data

In [6]:
# 4. summing up b.a.d MI for each residue using dictionaries

file4 = open('/Users/psz898/OneDrive - Københavns Universitet/Documents/SCIENCE/UCPH/info_comp/parent_bb_test/input/PARENT-master_shuf_MIST.txt', 'r') 
count = 0

# d b 824 714   MI: 0.00348862814858   left to process: 93
# TOTAL 2D BONDS-BONDS MUTUAL INFORMATION = 5.04769440071
# TOTAL 2D BONDS-ANGLES MUTUAL INFORMATION = 3.01188561498
# TOTAL 2D BONDS-DIHEDRALS MUTUAL INFORMATION = 4.47300695404
# TOTAL 2D ANGLES-ANGLES MUTUAL INFORMATION = 25.6458029556
# TOTAL 2D ANGLES-DIHEDRALS MUTUAL INFORMATION = 48.0856928134
# TOTAL 2D DIHEDRALS-DIHEDRALS MUTUAL INFORMATION = 125.402388931

mi_bb = re.compile("^(b) (b)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")

mi_ba = re.compile("^(b) (a)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")
mi_ab = re.compile("^(a) (b)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")

mi_bd = re.compile("^(b) (d)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")
mi_db = re.compile("^(d) (b)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")

mi_aa = re.compile("^(a) (a)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")

mi_ad = re.compile("^(a) (d)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")
mi_da = re.compile("^(d) (a)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")

mi_dd = re.compile("^(d) (d)\s+(\d+)\s+(\d+)\s+MI:\s+(-?\d+.\d+)\s+left to process:\s+\d+\n")


bb_mi_res_all = {}
bb_mi_res_bb = {}
bb_mi_res_rest = {}

ba_mi_res_all = {}
ba_mi_res_bb = {}
ba_mi_res_rest = {}

bd_mi_res_all = {}
bd_mi_res_bb = {}
bd_mi_res_rest = {}

aa_mi_res_all = {}
aa_mi_res_bb = {}
aa_mi_res_rest = {}

ad_mi_res_all = {}
ad_mi_res_bb = {}
ad_mi_res_rest = {}

dd_mi_res_all = {}
dd_mi_res_bb = {}
dd_mi_res_rest = {}

while True: 
    count += 1
  
    # Get next line from file 
    line = file4.readline() 
  
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
        
    result_bb = mi_bb.search(line)

    result_ba = mi_ba.search(line)
    result_ab = mi_ab.search(line)
    
    result_bd = mi_bd.search(line)
    result_db = mi_db.search(line)

    result_aa = mi_aa.search(line)

    result_ad = mi_ad.search(line)
    result_da = mi_da.search(line)

    result_dd = mi_dd.search(line)

    # bond-bond 
    if result_bb:
        some_resA = bonds_res_dict[int(result_bb.group(3))]
        some_resB = bonds_res_dict[int(result_bb.group(4))]
        bb_mi_res_all.setdefault(some_resA, []).append(float(result_bb.group(5))/2)   
        bb_mi_res_all.setdefault(some_resB, []).append(float(result_bb.group(5))/2)   
    if result_bb and int(result_bb.group(3)) in np.bbB and int(result_bb.group(4)) in np.bbB:    
        some_resA = bonds_res_dict[int(result_bb.group(3))]
        some_resB = bonds_res_dict[int(result_bb.group(4))]
        bb_mi_res_bb.setdefault(some_resA, []).append(float(result_bb.group(5))/2)   
        bb_mi_res_bb.setdefault(some_resB, []).append(float(result_bb.group(5))/2) 
    if (result_bb and int(result_bb.group(3)) not in np.bbB) and (result_bb and int(result_bb.group(4)) not in np.bbB): 
        some_resA = bonds_res_dict[int(result_bb.group(3))]
        some_resB = bonds_res_dict[int(result_bb.group(4))]
        bb_mi_res_rest.setdefault(some_resA, []).append(float(result_bb.group(5))/2)   
        bb_mi_res_rest.setdefault(some_resB, []).append(float(result_bb.group(5))/2)         
    if (result_bb and int(result_bb.group(3)) not in np.bbB) and (result_bb and int(result_bb.group(4)) in np.bbB): 
        some_resA = bonds_res_dict[int(result_bb.group(3))]
        some_resB = bonds_res_dict[int(result_bb.group(4))]
        bb_mi_res_rest.setdefault(some_resA, []).append(float(result_bb.group(5))/2)   
        bb_mi_res_bb.setdefault(some_resB, []).append(float(result_bb.group(5))/2) 
    if (result_bb and int(result_bb.group(3)) in np.bbB) and (result_bb and int(result_bb.group(4)) not in np.bbB): 
        some_resA = bonds_res_dict[int(result_bb.group(3))]
        some_resB = bonds_res_dict[int(result_bb.group(4))]
        bb_mi_res_bb.setdefault(some_resA, []).append(float(result_bb.group(5))/2)   
        bb_mi_res_rest.setdefault(some_resB, []).append(float(result_bb.group(5))/2) 

       
    # bond-angle
    if result_ba:
        some_resA = bonds_res_dict[int(result_ba.group(3))]
        some_resB = angles_res_dict[int(result_ba.group(4))]
        ba_mi_res_all.setdefault(some_resA, []).append(float(result_ba.group(5))/2)   
        ba_mi_res_all.setdefault(some_resB, []).append(float(result_ba.group(5))/2) 
    if result_ba and int(result_ba.group(3)) in np.bbB and int(result_ba.group(4)) in np.bbA: 
        some_resA = bonds_res_dict[int(result_ba.group(3))]
        some_resB = angles_res_dict[int(result_ba.group(4))]
        ba_mi_res_bb.setdefault(some_resA, []).append(float(result_ba.group(5))/2)   
        ba_mi_res_bb.setdefault(some_resB, []).append(float(result_ba.group(5))/2) 
    if (result_ba and int(result_ba.group(3)) not in np.bbB) and (result_ba and int(result_ba.group(4)) not in np.bbA):
        some_resA = bonds_res_dict[int(result_ba.group(3))]
        some_resB = angles_res_dict[int(result_ba.group(4))]
        ba_mi_res_rest.setdefault(some_resA, []).append(float(result_ba.group(5))/2)   
        ba_mi_res_rest.setdefault(some_resB, []).append(float(result_ba.group(5))/2)    
    if (result_ba and int(result_ba.group(3)) not in np.bbB) and (result_ba and int(result_ba.group(4)) in np.bbA):
        some_resA = bonds_res_dict[int(result_ba.group(3))]
        some_resB = angles_res_dict[int(result_ba.group(4))]
        ba_mi_res_rest.setdefault(some_resA, []).append(float(result_ba.group(5))/2)   
        ba_mi_res_bb.setdefault(some_resB, []).append(float(result_ba.group(5))/2)   
    if (result_ba and int(result_ba.group(3)) in np.bbB) and (result_ba and int(result_ba.group(4)) not in np.bbA):
        some_resA = bonds_res_dict[int(result_ba.group(3))]
        some_resB = angles_res_dict[int(result_ba.group(4))]
        ba_mi_res_bb.setdefault(some_resA, []).append(float(result_ba.group(5))/2)   
        ba_mi_res_rest.setdefault(some_resB, []).append(float(result_ba.group(5))/2)       
    # angle-bond (add to common bond-angle list)
    if result_ab:
        some_resA = angles_res_dict[int(result_ab.group(3))]
        some_resB = bonds_res_dict[int(result_ab.group(4))]
        ba_mi_res_all.setdefault(some_resA, []).append(float(result_ab.group(5))/2)   
        ba_mi_res_all.setdefault(some_resB, []).append(float(result_ab.group(5))/2) 
    if result_ab and int(result_ab.group(3)) in np.bbA and int(result_ab.group(4)) in np.bbB:
        some_resA = angles_res_dict[int(result_ab.group(3))]
        some_resB = bonds_res_dict[int(result_ab.group(4))]
        ba_mi_res_bb.setdefault(some_resA, []).append(float(result_ab.group(5))/2)   
        ba_mi_res_bb.setdefault(some_resB, []).append(float(result_ab.group(5))/2) 
    if (result_ab and int(result_ab.group(3)) not in np.bbA) and (result_ab and int(result_ab.group(4)) not in np.bbB):
        some_resA = angles_res_dict[int(result_ab.group(3))]
        some_resB = bonds_res_dict[int(result_ab.group(4))]
        ba_mi_res_rest.setdefault(some_resA, []).append(float(result_ab.group(5))/2)   
        ba_mi_res_rest.setdefault(some_resB, []).append(float(result_ab.group(5))/2)        
    if (result_ab and int(result_ab.group(3)) not in np.bbA) and (result_ab and int(result_ab.group(4)) in np.bbB):
        some_resA = angles_res_dict[int(result_ab.group(3))]
        some_resB = bonds_res_dict[int(result_ab.group(4))]
        ba_mi_res_rest.setdefault(some_resA, []).append(float(result_ab.group(5))/2)   
        ba_mi_res_bb.setdefault(some_resB, []).append(float(result_ab.group(5))/2)   
    if (result_ab and int(result_ab.group(3)) in np.bbA) and (result_ab and int(result_ab.group(4)) not in np.bbB):
        some_resA = angles_res_dict[int(result_ab.group(3))]
        some_resB = bonds_res_dict[int(result_ab.group(4))]
        ba_mi_res_bb.setdefault(some_resA, []).append(float(result_ab.group(5))/2)   
        ba_mi_res_rest.setdefault(some_resB, []).append(float(result_ab.group(5))/2)    
             
    
    # bond-dihedral
    if result_bd:
        some_resA = bonds_res_dict[int(result_bd.group(3))]
        some_resB = dihedrals_res_dict[int(result_bd.group(4))]
        bd_mi_res_all.setdefault(some_resA, []).append(float(result_bd.group(5))/2)   
        bd_mi_res_all.setdefault(some_resB, []).append(float(result_bd.group(5))/2) 
    if result_bd and int(result_bd.group(3)) in np.bbB and int(result_bd.group(4)) in np.bbD:
        some_resA = bonds_res_dict[int(result_bd.group(3))]
        some_resB = dihedrals_res_dict[int(result_bd.group(4))]
        bd_mi_res_bb.setdefault(some_resA, []).append(float(result_bd.group(5))/2)   
        bd_mi_res_bb.setdefault(some_resB, []).append(float(result_bd.group(5))/2) 
    if (result_bd and int(result_bd.group(3)) not in np.bbB) and (result_bd and int(result_bd.group(4)) not in np.bbD):
        some_resA = bonds_res_dict[int(result_bd.group(3))]
        some_resB = dihedrals_res_dict[int(result_bd.group(4))]
        bd_mi_res_rest.setdefault(some_resA, []).append(float(result_bd.group(5))/2)   
        bd_mi_res_rest.setdefault(some_resB, []).append(float(result_bd.group(5))/2) 
    if (result_bd and int(result_bd.group(3)) not in np.bbB) and (result_bd and int(result_bd.group(4)) in np.bbD):
        some_resA = bonds_res_dict[int(result_bd.group(3))]
        some_resB = dihedrals_res_dict[int(result_bd.group(4))]
        bd_mi_res_rest.setdefault(some_resA, []).append(float(result_bd.group(5))/2)   
        bd_mi_res_bb.setdefault(some_resB, []).append(float(result_bd.group(5))/2)
    if (result_bd and int(result_bd.group(3)) in np.bbB) and (result_bd and int(result_bd.group(4)) not in np.bbD):
        some_resA = bonds_res_dict[int(result_bd.group(3))]
        some_resB = dihedrals_res_dict[int(result_bd.group(4))]
        bd_mi_res_bb.setdefault(some_resA, []).append(float(result_bd.group(5))/2)   
        bd_mi_res_rest.setdefault(some_resB, []).append(float(result_bd.group(5))/2)
 
    # dihedral-bond (add to common bond-dihedral list) 
    if result_db:
        some_resA = dihedrals_res_dict[int(result_db.group(3))]
        some_resB = bonds_res_dict[int(result_db.group(4))]
        bd_mi_res_all.setdefault(some_resA, []).append(float(result_db.group(5))/2)   
        bd_mi_res_all.setdefault(some_resB, []).append(float(result_db.group(5))/2) 
    if result_db and int(result_db.group(3)) in np.bbD and int(result_db.group(4)) in np.bbB:
        some_resA = dihedrals_res_dict[int(result_db.group(3))]
        some_resB = bonds_res_dict[int(result_db.group(4))]
        bd_mi_res_bb.setdefault(some_resA, []).append(float(result_db.group(5))/2)   
        bd_mi_res_bb.setdefault(some_resB, []).append(float(result_db.group(5))/2) 
    if (result_db and int(result_db.group(3)) not in np.bbD) and (result_db and int(result_db.group(4)) not in np.bbB):
        some_resA = dihedrals_res_dict[int(result_db.group(3))]
        some_resB = bonds_res_dict[int(result_db.group(4))]
        bd_mi_res_rest.setdefault(some_resA, []).append(float(result_db.group(5))/2)   
        bd_mi_res_rest.setdefault(some_resB, []).append(float(result_db.group(5))/2) 
    if (result_db and int(result_db.group(3)) not in np.bbD) and (result_db and int(result_db.group(4)) in np.bbB):
        some_resA = dihedrals_res_dict[int(result_db.group(3))]
        some_resB = bonds_res_dict[int(result_db.group(4))]
        bd_mi_res_rest.setdefault(some_resA, []).append(float(result_db.group(5))/2)   
        bd_mi_res_bb.setdefault(some_resB, []).append(float(result_db.group(5))/2) 
    if (result_db and int(result_db.group(3)) in np.bbD) and (result_db and int(result_db.group(4)) not in np.bbB):  
        some_resA = dihedrals_res_dict[int(result_db.group(3))]
        some_resB = bonds_res_dict[int(result_db.group(4))]
        bd_mi_res_bb.setdefault(some_resA, []).append(float(result_db.group(5))/2)   
        bd_mi_res_rest.setdefault(some_resB, []).append(float(result_db.group(5))/2) 

    # angle-angle 
    if result_aa:
        some_resA = angles_res_dict[int(result_aa.group(3))]
        some_resB = angles_res_dict[int(result_aa.group(4))]
        aa_mi_res_all.setdefault(some_resA, []).append(float(result_aa.group(5))/2)   
        aa_mi_res_all.setdefault(some_resB, []).append(float(result_aa.group(5))/2)
    if result_aa and int(result_aa.group(3)) in np.bbA and int(result_aa.group(4)) in np.bbA:
        some_resA = angles_res_dict[int(result_aa.group(3))]
        some_resB = angles_res_dict[int(result_aa.group(4))]
        aa_mi_res_bb.setdefault(some_resA, []).append(float(result_aa.group(5))/2)   
        aa_mi_res_bb.setdefault(some_resB, []).append(float(result_aa.group(5))/2)
    if (result_aa and int(result_aa.group(3)) not in np.bbA) and (result_aa and int(result_aa.group(4)) not in np.bbA):
        some_resA = angles_res_dict[int(result_aa.group(3))]
        some_resB = angles_res_dict[int(result_aa.group(4))]
        aa_mi_res_rest.setdefault(some_resA, []).append(float(result_aa.group(5))/2)   
        aa_mi_res_rest.setdefault(some_resB, []).append(float(result_aa.group(5))/2)
    if (result_aa and int(result_aa.group(3)) not in np.bbA) and (result_aa and int(result_aa.group(4)) in np.bbA):
        some_resA = angles_res_dict[int(result_aa.group(3))]
        some_resB = angles_res_dict[int(result_aa.group(4))]
        aa_mi_res_rest.setdefault(some_resA, []).append(float(result_aa.group(5))/2)   
        aa_mi_res_bb.setdefault(some_resB, []).append(float(result_aa.group(5))/2)
    if (result_aa and int(result_aa.group(3)) in np.bbA) and (result_aa and int(result_aa.group(4)) not in np.bbA):
        some_resA = angles_res_dict[int(result_aa.group(3))]
        some_resB = angles_res_dict[int(result_aa.group(4))]
        aa_mi_res_bb.setdefault(some_resA, []).append(float(result_aa.group(5))/2)   
        aa_mi_res_rest.setdefault(some_resB, []).append(float(result_aa.group(5))/2)
    
        
    # angle-dihedral
    if result_ad:
        some_resA = angles_res_dict[int(result_ad.group(3))]
        some_resB = dihedrals_res_dict[int(result_ad.group(4))]
        ad_mi_res_all.setdefault(some_resA, []).append(float(result_ad.group(5))/2)   
        ad_mi_res_all.setdefault(some_resB, []).append(float(result_ad.group(5))/2)
    if result_ad and int(result_ad.group(3)) in np.bbA and int(result_ad.group(4)) in np.bbD:
        some_resA = angles_res_dict[int(result_ad.group(3))]
        some_resB = dihedrals_res_dict[int(result_ad.group(4))]
        ad_mi_res_bb.setdefault(some_resA, []).append(float(result_ad.group(5))/2)   
        ad_mi_res_bb.setdefault(some_resB, []).append(float(result_ad.group(5))/2)
    if (result_ad and int(result_ad.group(3)) not in np.bbA) and (result_ad and int(result_ad.group(4)) not in np.bbD):
        some_resA = angles_res_dict[int(result_ad.group(3))]
        some_resB = dihedrals_res_dict[int(result_ad.group(4))]
        ad_mi_res_rest.setdefault(some_resA, []).append(float(result_ad.group(5))/2)   
        ad_mi_res_rest.setdefault(some_resB, []).append(float(result_ad.group(5))/2)
    if (result_ad and int(result_ad.group(3)) not in np.bbA) and (result_ad and int(result_ad.group(4)) in np.bbD):
        some_resA = angles_res_dict[int(result_ad.group(3))]
        some_resB = dihedrals_res_dict[int(result_ad.group(4))]
        ad_mi_res_rest.setdefault(some_resA, []).append(float(result_ad.group(5))/2)   
        ad_mi_res_bb.setdefault(some_resB, []).append(float(result_ad.group(5))/2)
    if (result_ad and int(result_ad.group(3)) in np.bbA) and (result_ad and int(result_ad.group(4)) not in np.bbD):
        some_resA = angles_res_dict[int(result_ad.group(3))]
        some_resB = dihedrals_res_dict[int(result_ad.group(4))]
        ad_mi_res_bb.setdefault(some_resA, []).append(float(result_ad.group(5))/2)   
        ad_mi_res_rest.setdefault(some_resB, []).append(float(result_ad.group(5))/2)
    
    # dihedral-angle (add to common angle-dihedral list)
    if result_da:
        some_resA = dihedrals_res_dict[int(result_da.group(3))]
        some_resB = angles_res_dict[int(result_da.group(4))]
        ad_mi_res_all.setdefault(some_resA, []).append(float(result_da.group(5))/2)   
        ad_mi_res_all.setdefault(some_resB, []).append(float(result_da.group(5))/2)
    if result_da and int(result_da.group(3)) in np.bbD and int(result_da.group(4)) in np.bbA:
        some_resA = dihedrals_res_dict[int(result_da.group(3))]
        some_resB = angles_res_dict[int(result_da.group(4))]
        ad_mi_res_bb.setdefault(some_resA, []).append(float(result_da.group(5))/2)   
        ad_mi_res_bb.setdefault(some_resB, []).append(float(result_da.group(5))/2)
    if (result_da and int(result_da.group(3)) not in np.bbD) and (result_da and int(result_da.group(4)) not in np.bbA): 
        some_resA = dihedrals_res_dict[int(result_da.group(3))]
        some_resB = angles_res_dict[int(result_da.group(4))]
        ad_mi_res_rest.setdefault(some_resA, []).append(float(result_da.group(5))/2)   
        ad_mi_res_rest.setdefault(some_resB, []).append(float(result_da.group(5))/2)
    if (result_da and int(result_da.group(3)) not in np.bbD) and (result_da and int(result_da.group(4)) in np.bbA):
        some_resA = dihedrals_res_dict[int(result_da.group(3))]
        some_resB = angles_res_dict[int(result_da.group(4))]
        ad_mi_res_rest.setdefault(some_resA, []).append(float(result_da.group(5))/2)   
        ad_mi_res_bb.setdefault(some_resB, []).append(float(result_da.group(5))/2)
    if (result_da and int(result_da.group(3)) in np.bbD) and (result_da and int(result_da.group(4)) not in np.bbA):
        some_resA = dihedrals_res_dict[int(result_da.group(3))]
        some_resB = angles_res_dict[int(result_da.group(4))]
        ad_mi_res_bb.setdefault(some_resA, []).append(float(result_da.group(5))/2)   
        ad_mi_res_rest.setdefault(some_resB, []).append(float(result_da.group(5))/2)
        
        
    # dihedral-dihedral    
    if result_dd:
        some_resA = angles_res_dict[int(result_dd.group(3))]
        some_resB = angles_res_dict[int(result_dd.group(4))]
        dd_mi_res_all.setdefault(some_resA, []).append(float(result_dd.group(5))/2)   
        dd_mi_res_all.setdefault(some_resB, []).append(float(result_dd.group(5))/2)
    if result_dd and int(result_dd.group(3)) in np.bbD and int(result_dd.group(4)) in np.bbD:
        some_resA = angles_res_dict[int(result_dd.group(3))]
        some_resB = angles_res_dict[int(result_dd.group(4))]
        dd_mi_res_bb.setdefault(some_resA, []).append(float(result_dd.group(5))/2)   
        dd_mi_res_bb.setdefault(some_resB, []).append(float(result_dd.group(5))/2)
    if (result_dd and int(result_dd.group(3)) not in np.bbD) and (result_dd and int(result_dd.group(4)) not in np.bbD):
        some_resA = angles_res_dict[int(result_dd.group(3))]
        some_resB = angles_res_dict[int(result_dd.group(4))]
        dd_mi_res_rest.setdefault(some_resA, []).append(float(result_dd.group(5))/2)   
        dd_mi_res_rest.setdefault(some_resB, []).append(float(result_dd.group(5))/2)
    if (result_dd and int(result_dd.group(3)) not in np.bbD) and (result_dd and int(result_dd.group(4)) in np.bbD):
        some_resA = angles_res_dict[int(result_dd.group(3))]
        some_resB = angles_res_dict[int(result_dd.group(4))]
        dd_mi_res_rest.setdefault(some_resA, []).append(float(result_dd.group(5))/2)   
        dd_mi_res_bb.setdefault(some_resB, []).append(float(result_dd.group(5))/2)
    if (result_dd and int(result_dd.group(3)) in np.bbD) and (result_dd and int(result_dd.group(4)) not in np.bbD):
        some_resA = angles_res_dict[int(result_dd.group(3))]
        some_resB = angles_res_dict[int(result_dd.group(4))]
        dd_mi_res_bb.setdefault(some_resA, []).append(float(result_dd.group(5))/2)   
        dd_mi_res_rest.setdefault(some_resB, []).append(float(result_dd.group(5))/2)
       
         
file4.close 

# sum MI per residue and total sum

bb_mi_res_all_sum = {k: sum(bb_mi_res_all[k]) for k in bb_mi_res_all.keys()}
bb_mi_res_all_total_sum = sum(bb_mi_res_all_sum.values())
bb_mi_res_bb_sum = {k: sum(bb_mi_res_bb[k]) for k in bb_mi_res_bb.keys()}
bb_mi_res_bb_total_sum = sum(bb_mi_res_bb_sum.values())
bb_mi_res_rest_sum = {k: sum(bb_mi_res_rest[k]) for k in bb_mi_res_rest.keys()}
bb_mi_res_rest_total_sum = sum(bb_mi_res_rest_sum.values())

ba_mi_res_all_sum = {k: sum(ba_mi_res_all[k]) for k in ba_mi_res_all.keys()}
ba_mi_res_all_total_sum = sum(ba_mi_res_all_sum.values())
ba_mi_res_bb_sum = {k: sum(ba_mi_res_bb[k]) for k in ba_mi_res_bb.keys()}
ba_mi_res_bb_total_sum = sum(ba_mi_res_bb_sum.values())
ba_mi_res_rest_sum = {k: sum(ba_mi_res_rest[k]) for k in ba_mi_res_rest.keys()}
ba_mi_res_rest_total_sum = sum(ba_mi_res_rest_sum.values())

bd_mi_res_all_sum = {k: sum(bd_mi_res_all[k]) for k in bd_mi_res_all.keys()}
bd_mi_res_all_total_sum = sum(bd_mi_res_all_sum.values())
bd_mi_res_bb_sum = {k: sum(bd_mi_res_bb[k]) for k in bd_mi_res_bb.keys()}
bd_mi_res_bb_total_sum = sum(bd_mi_res_bb_sum.values())
bd_mi_res_rest_sum = {k: sum(bd_mi_res_rest[k]) for k in bd_mi_res_rest.keys()}
bd_mi_res_rest_total_sum = sum(bd_mi_res_rest_sum.values())

aa_mi_res_all_sum = {k: sum(aa_mi_res_all[k]) for k in aa_mi_res_all.keys()}
aa_mi_res_all_total_sum = sum(aa_mi_res_all_sum.values())
aa_mi_res_bb_sum = {k: sum(aa_mi_res_bb[k]) for k in aa_mi_res_bb.keys()}
aa_mi_res_bb_total_sum = sum(aa_mi_res_bb_sum.values())
aa_mi_res_rest_sum = {k: sum(aa_mi_res_rest[k]) for k in aa_mi_res_rest.keys()}
aa_mi_res_rest_total_sum = sum(aa_mi_res_rest_sum.values())

ad_mi_res_all_sum = {k: sum(ad_mi_res_all[k]) for k in ad_mi_res_all.keys()}
ad_mi_res_all_total_sum = sum(ad_mi_res_all_sum.values())
ad_mi_res_bb_sum = {k: sum(ad_mi_res_bb[k]) for k in ad_mi_res_bb.keys()}
ad_mi_res_bb_total_sum = sum(ad_mi_res_bb_sum.values())
ad_mi_res_rest_sum = {k: sum(ad_mi_res_rest[k]) for k in ad_mi_res_rest.keys()}
ad_mi_res_rest_total_sum = sum(ad_mi_res_rest_sum.values())

dd_mi_res_all_sum = {k: sum(dd_mi_res_all[k]) for k in dd_mi_res_all.keys()}
dd_mi_res_all_total_sum = sum(dd_mi_res_all_sum.values())
dd_mi_res_bb_sum = {k: sum(dd_mi_res_bb[k]) for k in dd_mi_res_bb.keys()}
dd_mi_res_bb_total_sum = sum(dd_mi_res_bb_sum.values())
dd_mi_res_rest_sum = {k: sum(dd_mi_res_rest[k]) for k in dd_mi_res_rest.keys()}
dd_mi_res_rest_total_sum = sum(dd_mi_res_rest_sum.values())

sumBB = bonds_entropy_res_bb_total_sum+angles_entropy_res_bb_total_sum+dihedrals_entropy_res_bb_total_sum-bb_mi_res_bb_total_sum-ba_mi_res_bb_total_sum-bd_mi_res_bb_total_sum-aa_mi_res_bb_total_sum-ad_mi_res_bb_total_sum-dd_mi_res_bb_total_sum
sumRest = bonds_entropy_res_rest_total_sum+angles_entropy_res_rest_total_sum+dihedrals_entropy_res_rest_total_sum-bb_mi_res_rest_total_sum-ba_mi_res_rest_total_sum-bd_mi_res_rest_total_sum-aa_mi_res_rest_total_sum-ad_mi_res_rest_total_sum-dd_mi_res_rest_total_sum
sumAll = bonds_entropy_res_all_total_sum+angles_entropy_res_all_total_sum+dihedrals_entropy_res_all_total_sum-bb_mi_res_all_total_sum-ba_mi_res_all_total_sum-bd_mi_res_all_total_sum-aa_mi_res_all_total_sum-ad_mi_res_all_total_sum-dd_mi_res_all_total_sum


# per residue total entropy (including MI):

all_dict_bb =[bonds_entropy_res_bb_sum,angles_entropy_res_bb_sum,dihedrals_entropy_res_bb_sum, negativedict(bb_mi_res_bb_sum),negativedict(ba_mi_res_bb_sum),negativedict(bd_mi_res_bb_sum),negativedict(aa_mi_res_bb_sum),negativedict(ad_mi_res_bb_sum),negativedict(dd_mi_res_bb_sum)]
all_dict_rest =[bonds_entropy_res_rest_sum,angles_entropy_res_rest_sum,dihedrals_entropy_res_rest_sum, negativedict(bb_mi_res_rest_sum),negativedict(ba_mi_res_rest_sum),negativedict(bd_mi_res_rest_sum),negativedict(aa_mi_res_rest_sum),negativedict(ad_mi_res_rest_sum),negativedict(dd_mi_res_rest_sum)]
all_dict_all =[bonds_entropy_res_all_sum,angles_entropy_res_all_sum,dihedrals_entropy_res_all_sum, negativedict(bb_mi_res_all_sum),negativedict(ba_mi_res_all_sum),negativedict(bd_mi_res_all_sum),negativedict(aa_mi_res_all_sum),negativedict(ad_mi_res_all_sum),negativedict(dd_mi_res_all_sum)]

# sum all dictionaries; negativedict(bb_mi_res_bb_sum) is used to sum entropy with negative of MI (istead of subtraction of MI)

counter_bb = collections.Counter()
for d in all_dict_bb: 
    counter_bb.update(d)      
sum_counter_bb = sum(counter_bb.values()) 
counter_bb = dict(counter_bb) # convert to a regular dictionary

counter_rest = collections.Counter()
for d in all_dict_rest: 
    counter_rest.update(d)      
sum_counter_rest = sum(counter_rest.values()) 
counter_rest = dict(counter_rest) # convert to a regular dictionary

counter_all = collections.Counter()
for d in all_dict_all: 
    counter_all.update(d)      
sum_counter_all = sum(counter_all.values()) 
counter_all = dict(counter_all) # convert to a regular dictionary

print("bb")
for k,v in sorted(counter_bb.items()):
    print(k, v)
print("rest")    
for k,v in sorted(counter_rest.items()):
    print(k, v)
print("all")    
for k,v in sorted(counter_all.items()):
    print(k, v)


#print (sum_counter_bb)
#print (counter_bb)

#print (sum_counter_rest)
#print (counter_rest)

#print (sum_counter_all)
#print (counter_all)


# print("")
# print ("bond-bond mi bb:", bb_mi_res_bb_total_sum)
# print ("bond-bond mi rest:", bb_mi_res_rest_total_sum)
# print ("bond-bond mi all:", bb_mi_res_all_total_sum)
# print("")
# print ("bond-angle mi bb:", ba_mi_res_bb_total_sum)
# print ("bond-angle mi rest:", ba_mi_res_rest_total_sum)
# print ("bond-angle mi all:", ba_mi_res_all_total_sum)
# print("")
# print ("bond-angle mi bb:", bd_mi_res_bb_total_sum)
# print ("bond-angle mi rest:", bd_mi_res_rest_total_sum)
# print ("bond-angle mi all:", bd_mi_res_all_total_sum)
# print("")
# print ("angle-angle mi bb:", aa_mi_res_bb_total_sum)
# print ("angle-angle mi rest:", aa_mi_res_rest_total_sum)
# print ("angle-angle mi all:", aa_mi_res_all_total_sum)
# print("")
# print ("angle-dihedral mi bb:", ad_mi_res_bb_total_sum)
# print ("angle-dihedral mi rest:", ad_mi_res_rest_total_sum)
# print ("angle-dihedral mi all:", ad_mi_res_all_total_sum)
# print("")
# print ("dihedral-dihedral mi bb:", dd_mi_res_bb_total_sum)
# print ("dihedral-dihedral mi rest:", dd_mi_res_rest_total_sum)
# print ("dihedral-dihedral mi all:", dd_mi_res_all_total_sum)
print("")
print ("total entropy bb:",sumBB)
print ("total entropy rest:",sumRest)
print ("total entropy all:",sumAll)
print("")


# sum all dictionaries in the list dict1 without using collections.Counter()
# final = {}
# for k in dict1[0].keys():           # Init all elements to zero.
#     final[k] = 0
# for d in dict1:
#     for k in d.keys():
#         final[k] = final[k] + d[k]  # Update the element.
# final_sum = sum(final.values())

bb
1 -26.080456226528423
2 -20.063984861881778
3 -21.04144034027475
4 -21.915959203977067
5 -21.84828804736741
6 -16.34452971122939
7 -21.40466601873365
8 -21.51021995743268
9 -21.772638329963115
10 -21.289830162319774
11 -21.085148642148482
12 -21.260778864932032
13 -21.776155352985796
14 -21.43660186413376
15 -21.20343074778118
16 -21.42309634387701
17 -21.636600314234506
18 -20.963921194197457
19 -21.224332957219495
20 -22.029224233226177
21 -21.59219767955543
22 -21.095575741651267
23 -20.89366348891123
24 -21.34504440985669
25 -16.059799536895166
26 -20.47178094871195
27 -20.595267739051682
28 -20.686376697104077
29 -20.768964176516302
30 -21.138877064772796
31 -21.556729636928093
32 -21.69551000428694
33 -15.99779865369027
34 -20.625156906196725
35 -19.179683645186344
36 -19.572010862402017
37 -19.707116525339117
38 -20.039712230271107
39 -19.884526042394306
40 -20.48995563207895
41 -17.5022470044946
42 -8.811932569781856
43 -20.68902278446157
44 -20.643737196365255
45 -20.671639